In [ ]:
from functools import partial
from pathlib import Path

import pandas as pd
import geopandas as gdp
from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as sm
import ipyvuetify as v

from utils import messages as ms
from utils import parameters as pm
from scripts import table as run

In [ ]:
#input and output as mutable objects 
class Table_io:
    def __init__(self):
        #input 
        self.file = None
        self.lng = None
        self.lat = None
        self.id = None
        
        #output
        self.pts = None
        
tb_io = Table_io()

In [ ]:
#output to display messages
file_output = sw.Alert().add_msg(ms.TABLE_INTRO)

#btn 
file_btn = sw.Btn(ms.TABLE_BTN, icon='mdi-check')

#create the inputs
file_select = sw.FileInput(['.csv', '.txt'])
file_select.bind_io(file_output, tb_io, 'file')

#create a file selector 
id_ = 'file_widget'
title = "Select File"

file_tile = sw.Tile(
    id_,
    title,
    btn=file_btn, 
    inputs=[file_select],
    output=file_output
)


In [ ]:
#tile to select columns 
col_output = sw.Alert().add_msg(ms.COL_INTRO)

#btn 
col_btn = sw.Btn(ms.COL_BTN, icon='mdi-check')

#create the inputs
col_lng = v.Select(items=[], label=ms.COL_LNG, v_model=None)
col_lat = v.Select(items=[], label=ms.COL_LAT, v_model=None)
col_id = v.Select(items=[], label=ms.COL_ID, v_model=None)

#bind the inputs 
col_output.bind(col_lng, tb_io, 'lng')
col_output.bind(col_lat, tb_io, 'lat')
col_output.bind(col_id, tb_io, 'id')

#create a file selector 
id_ = 'file_widget'
title = "Select Columns"

col_tile = sw.Tile(
    id_,
    title,
    btn=col_btn, 
    inputs=[col_lng, col_lat, col_id],
    output=col_output
)

In [ ]:
#tiles to display the points on a map
tb_map = sm.SepalMap(['CartoDB.Positron'])
title = 'Display points'
tb_map_tile = sw.Tile(
    id_,
    title,
    inputs=[tb_map]
)

In [ ]:
#function to actualise the values of the select column tile 
def update_col(cols, values):
    for col in cols:
        col.items = values
        col.v_model=None

In [ ]:
#validate the points selection 
def load_file(widget, event, data, output, cols):
    
    #toggle the loading button
    widget.toggle_loading()
    
    #load the inputs 
    file = getattr(tb_io, 'file')
    
    #check input
    if not output.check_input(file, ms.NOT_A_FILE): return widget.toggle_loading()
    
    #check the file
    error = run.isConform(file)
    if error:
        output.add_live_msg(error, 'error')
        return widget.toggle_loading()
    
    #read the columns names
    sep = ',' if Path(file).suffix == '.csv' else ' '
    df = pd.read_csv(file, sep=sep)
    
    update_col(cols, df.columns.tolist())
    
    #write the values in the output
    output.add_msg(ms.READ_COLUMNS.format(', '.join(df.columns)), 'success')
    
    #toggle the loading button 
    widget.toggle_loading()
    
    return 

file_btn.on_event('click', partial(
    load_file,
    output = file_output,
    cols = [col_lng, col_lat, col_id]
))

In [ ]:
def load_columns(widget, event, data, output, m):
    
    #toggle the loading button 
    widget.toggle_loading()
    
    #define variable 
    file = getattr(tb_io, 'file')
    lat = getattr(tb_io, 'lat')
    lng = getattr(tb_io, 'lng')
    id_ = getattr(tb_io, 'id')
    
    
    #check the variables 
    if not output.check_input(lat, ms.MISSING_INPUT): return widget.toggle_input()
    if not output.check_input(lng, ms.MISSING_INPUT): return widget.toggle_input()
    if not output.check_input(id_, ms.MISSING_INPUT): return widget.toggle_input()    
    
    #verify that they are all unique
    if len(set([lat, lng, id_])) != len([lat, lng, id_]): 
        output.add_msg(ms.REPEATED_INPUT, 'error')
        return widget.toggle_loading()
    
    #create the pts geodataframe
    sep = ',' if Path(file).suffix == '.csv' else ' '
    df = pd.read_csv(file, sep=sep)
    df = df.filter(items=[lat, lng, id_])
    df = df.rename(columns={lat: 'lat', lng: 'lng', id_: 'id'})
    gdf = gdp.GeoDataFrame(df, geometry=gdp.points_from_xy(df.lng, df.lat), crs='EPSG:4326')
    
    #load the map
    run.setMap(gdf, m) 
    
    #set the dataframe in output 
    setattr(tb_io, 'pts', gdf)
    
    output.add_msg(ms.VALID_COLUMNS, 'success')
    
    #toggle the loading button 
    widget.toggle_loading()
    
    return 

col_btn.on_event('click', partial(
    load_columns,
    output = col_output,
    m = tb_map
))

In [ ]:
file_tile

In [ ]:
col_tile

In [ ]:
tb_map_tile